In [15]:
import numpy as np
from scipy.stats import norm

class ModelloBSM:

    #Modello di valutazione delle opzioni di Black-Scholes-Merton.

    #Attributi:
        #S: Prezzo iniziale dell'asset sottostante
        #K: Prezzo di esercizio dell'opzione
        #T: Tempo fino alla scadenza
        #r: Tasso di interesse privo di rischio
        #vol: Volatilità dell'asset sottostante
        #q: Dividendo continuo


    def __init__(self, S, K, T, r, vol, q=0):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.vol = vol
        self.q = q

    @staticmethod
    def N(x):
        #Funzione di distribuzione cumulativa della normale standard.
        return norm.cdf(x)

    def d1(self):
        #Calcola il parametro d1.
        return (np.log(self.S / self.K) + (self.r - self.q + self.vol**2 / 2) * self.T) / (
            self.vol * np.sqrt(self.T)
        )

    def d2(self):
        #Calcola il parametro d2.
        return self.d1() - self.vol * np.sqrt(self.T)

    def call_value(self):
        #Calcola il valore dell'opzione call.
        d1 = self.d1()
        d2 = self.d2()
        return self.S * np.exp(-self.q * self.T) * self.N(d1) - self.K * np.exp(-self.r * self.T) * self.N(d2)

    def put_value(self):
        #Calcola il valore dell'opzione put.
        d1 = self.d1()
        d2 = self.d2()
        return self.K * np.exp(-self.r * self.T) * self.N(-d2) - self.S * np.exp(-self.q * self.T) * self.N(-d1)

    def price(self, option_type='C'):
       #Calcola il prezzo dell'opzione.
            #option_type: 'C' per call, 'P' per put, 'B' per entrambi.
            #Prezzo dell'opzione o dizionario con i prezzi di call e put.

        if option_type == 'C':
            return self.call_value()
        elif option_type == 'P':
            return self.put_value()
        elif option_type == 'CP':
            return {'call': self.call_value(), 'put': self.put_value()}
        else:
            raise ValueError('Inserimento errato tipologia opzioni')


            #Greche
    def delta(self, option_type='C'):
        """Calcola il delta dell'opzione."""
        d1 = self.d1()
        if option_type == 'C':
            return np.exp(-self.q * self.T) * self.N(d1)
        elif option_type == 'P':
            return -np.exp(-self.q * self.T) * self.N(-d1)
        else:
            raise ValueError('Inserimento errato tipologia opzioni')

    def gamma(self):
        #Calcola il gamma dell'opzione.
        d1 = self.d1()
        return np.exp(-self.q * self.T) * norm.pdf(d1) / (self.S * self.vol * np.sqrt(self.T))

    def vega(self):
        #Calcola il vega dell'opzione."
        d1 = self.d1()
        return self.S * np.exp(-self.q * self.T) * norm.pdf(d1) * np.sqrt(self.T)

    def theta(self, option_type='C'):
        #Calcola il theta dell'opzione.
        d1 = self.d1()
        d2 = self.d2()
        if option_type == 'C':
            return -self.S * np.exp(-self.q * self.T) * norm.pdf(d1) * self.vol / (2 * np.sqrt(self.T)) \
                   - self.r * self.K * np.exp(-self.r * self.T) * self.N(d2) \
                   + self.q * self.S * np.exp(-self.q * self.T) * self.N(d1)
        elif option_type == 'P':
            return -self.S * np.exp(-self.q * self.T) * norm.pdf(d1) * self.vol / (2 * np.sqrt(self.T)) \
                   + self.r * self.K * np.exp(-self.r * self.T) * self.N(-d2) \
                   - self.q * self.S * np.exp(-self.q * self.T) * self.N(-d1)
        else:
            raise ValueError('Inserimento errato tipologia opzioni')

    def rho(self, option_type='C'):
        #Calcola il rho dell'opzione.
        d2 = self.d2()
        if option_type == 'C':
            return self.K * self.T * np.exp(-self.r * self.T) * self.N(d2)
        elif option_type == 'P':
            return -self.K * self.T * np.exp(-self.r * self.T) * self.N(-d2)
        else:
            raise ValueError('Inserimento errato tipologia opzioni')

if __name__ == '__main__':
    K = 23
    r = 0.042
    T = 2/12
    vol = 0.13935
    S = 23.95

    print("-----PREZZO OPZIONI-----")
    print(" ")
    print("     Call:",ModelloBSM(S, K, T, r, vol).price('C'),"    Put:",ModelloBSM(S, K, T, r, vol).price('P'))
    print(" ")
    print("-----VALORE GRECHE-----")
    print(" ")
    print("-Valore Delta:")
    print("     Call:",ModelloBSM(S, K, T, r, vol).delta('C'),"    Put:",ModelloBSM(S, K, T, r, vol).delta('P'))
    print(" ")
    print("-Valore Gamma:")
    print("     ",ModelloBSM(S, K, T, r, vol).gamma())
    print(" ")
    print("-Valore Vega:")
    print("     ",ModelloBSM(S, K, T, r, vol).vega())
    print(" ")
    print("-Valore Theta:")
    print("     ","Call:",ModelloBSM(S, K, T, r, vol).theta('C'),"    Put:",ModelloBSM(S, K, T, r, vol).theta('P'))
    print(" ")
    print("-Valore Rho:")
    print("     ","Call:",ModelloBSM(S, K, T, r, vol).rho('C'),"    Put:",ModelloBSM(S, K, T, r, vol).rho('P'))

-----PREZZO OPZIONI-----
 
     Call: 1.260844757321518     Put: 0.15040694478592975
 
-----VALORE GRECHE-----
 
-Valore Delta:
     Call: 0.8059156009125981     Put: -0.19408439908740188
 
-Valore Gamma:
      0.2017762392553335
 
-Valore Vega:
      2.688046526318949
 
-Valore Theta:
      Call: -1.881452873478115     Put: -0.9221912616046098
 
-Valore Rho:
      Call: 3.0068056474225338     Put: -0.7997880504882008
